In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install autogluon

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
import gc

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2021/train.csv').drop('id', axis=1)

In [ ]:
# reducing the train dataset but with equal target division
y = train['target']
train.drop('target',axis=1,inplace=True)
train_data,_,y_train,__ = train_test_split(train,y,train_size = 0.45,random_state=26, stratify=y)
train_data = pd.concat([train_data, y_train], axis=1, join="inner")
train_data.head()


In [ ]:
# deleting variables to save memory
del _
del __
del train
del y_train
gc.collect()   #freeing up the memory

In [ ]:
# We have nearly equal targets
print(train_data.shape)
train_data['target'].value_counts()

In [ ]:
metric = 'roc_auc'
time=7*3600
predictor = TabularPredictor(label='target', eval_metric=metric).fit(train_data= train_data, time_limit=time, presets='best_quality',excluded_model_types = ['KNN', 'XT' ,'RF'], verbosity=2,ag_args_fit={'num_gpus': 1})
predictor.leaderboard(train_data, silent=True)

In [ ]:
results = predictor.fit_summary()

In [ ]:
del train_data
gc.collect()

In [ ]:
test= pd.read_csv('/kaggle/input/tabular-playground-series-oct-2021/test.csv')
test = test.set_index('id')

In [ ]:
pred_probs = predictor.predict_proba(test_data)
pred_probs.head(5)

In [ ]:
pred_probs = pred_probs.set_index(test.index)

In [ ]:
sub_sample = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv', sep=',')
sub_sample = sub_sample.set_index('id')

In [ ]:
sub_sample['target']=pred_probs[1]

In [ ]:
sub_sample = sub_sample.reset_index()
sub_sample.to_csv('submission.csv',index=False)